## 음식 배달 시간 예측 
Predicting Food Delivery Time -Hackathon by IMS Proschool
- https://medium.com/code-to-express/predicting-food-delivery-time-hackathon-by-ims-proschool-744fd091390d


F1 Score: XGBoost, XGBoost with Normalization, XGBoost without TFIDF, Autogluon
- 0.27, 0.27, 0.20, 0.36
```
    "accuracy": 0.768565248738284,
    "macro avg": {
        "precision": 0.41980750041231374,
        "recall": 0.3505429827196643,
        "f1-score": 0.3699183299693503,
        "support": 2774
    },
```

In [1]:
! pip install xgboost
! pip install lightgbm

You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.
You should consider upgrading via the '/home/ec2-user/anaconda3/envs/python3/bin/python -m pip install --upgrade pip' command.


In [2]:
import os
import seaborn as sns
from scipy import stats
from tqdm import tqdm

import numpy as np
import pandas as pd
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from xgboost import XGBClassifier

import nltk
nltk.download('stopwords')
from nltk import word_tokenize
nltk.download('punkt')

import re
import string

from nltk.corpus import stopwords 
stop_words = stopwords.words('english')


import lightgbm as lgb
import matplotlib.pyplot as plt

%matplotlib inline

pd.options.display.max_columns = 100
from plotly import tools
import plotly.graph_objs as go
from plotly.offline import init_notebook_mode, iplot
import warnings
warnings.filterwarnings("ignore")

[nltk_data] Downloading package stopwords to
[nltk_data]     /home/ec2-user/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /home/ec2-user/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


## 데이터 로딩 및 간단 탐색

In [3]:
train_df = pd.read_excel('data/Data_Train_food.xlsx', encoding='latin-1')
test_df = pd.read_excel('data/Data_Test_food.xlsx', encoding='latin-1')
test_df

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews
0,ID_2842,"Mico Layout, Stage 2, BTM Layout,Bangalore","North Indian, Chinese, Assamese",₹350,₹50,4.2,361,225
1,ID_730,"Mico Layout, Stage 2, BTM Layout,Bangalore","Biryani, Kebab",₹100,₹50,NEW,-,-
2,ID_4620,"Sector 1, Noida",Fast Food,₹100,₹50,3.6,36,16
3,ID_5470,"Babarpur, New Delhi, Delhi","Mithai, North Indian, Chinese, Fast Food, Sout...",₹200,₹50,3.6,66,33
4,ID_3249,"Sector 1, Noida","Chinese, Fast Food",₹150,₹50,2.9,38,14
...,...,...,...,...,...,...,...,...
2769,ID_6342,Delhi University-GTB Nagar,"Fast Food, Chinese",₹100,₹50,NEW,-,-
2770,ID_8495,"Mico Layout, Stage 2, BTM Layout,Bangalore","Continental, North Indian, Desserts, Beverages",₹250,₹50,3.1,5,1
2771,ID_7122,"Noorkhan Bazaar, Malakpet, Hyderabad","Andhra, South Indian",₹150,₹50,3.0,16,1
2772,ID_2475,"D-Block, Sector 63, Noida",Bakery,₹100,₹99,3.0,7,2


In [4]:
pd.options.display.max_rows=5
train_df

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",₹200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",₹100,₹50,3.5,11,4,30 minutes
...,...,...,...,...,...,...,...,...,...
11092,ID_4115,"Sector 3, Marathalli",North Indian,₹100,₹50,3.1,24,9,30 minutes
11093,ID_4417,"Sector 63A,Gurgaon",North Indian,₹100,₹50,NEW,-,-,30 minutes


In [5]:
pd.options.display.max_rows=10
train_df.isnull().sum()

Restaurant       0
Location         0
Cuisines         0
Average_Cost     0
Minimum_Order    0
Rating           0
Votes            0
Reviews          0
Delivery_Time    0
dtype: int64

In [6]:
train_df.describe()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
count,11094,11094,11094,11094,11094,11094,11094,11094,11094
unique,7480,35,2179,26,18,33,1103,761,7
top,ID_7184,"Mico Layout, Stage 2, BTM Layout,Bangalore",North Indian,₹200,₹50,-,-,-,30 minutes
freq,22,947,850,3241,10118,1191,2074,2312,7406


## 데이타 클리닝

In [7]:
# #A function to find the maximun number of features in a single cell
# def max_features_in_single_row(train_df, test_df, delimiter):
#     max_info = 0 
#     item_lis = list(train_df.append(test_df))
#     for i in item_lis:
#         if len(i.split("{}".format(delimiter))) > max_info:
#             max_info = len(i.split("{}".format(delimiter)))
#             print("\n","-"*35)    
#             print("Max_Features in One Observation = ", max_info)
#         return max_info
    


In [8]:
# #This function splits a column in to n features where n is the maximum number of features in a single cell
# def feature_splitter(feat, name, delimiter, max_info):
#     item_lis = list()
#     extracted_features = {}
#     for i in range(max_info):
#         extracted_features['{}_Feature_{}'.format(name, i+1)] = []
#         print("-"*35)
#         print("Features Dictionary : ", extracted_features)
#   #tqdm is a graphics module that helps us see the progress bar
#     for i in tqdm(range(len(item_lis))):
#         for j in range(max_info):  
#             try:
#                 extracted_features['{}_Feature_{}'.format(name,j+1)].append(item_lis[i].split("{}".format(delimiter))[j].lower().strip())
#             except: 
#                 extracted_features['{}_Feature_{}'.format(name, j+1)].append(np.nan)

#     return extracted_features     

In [9]:
# max_features_in_single_row(train_df['Location'], test_df['Location'],',')

In [10]:
# max_features_in_single_row(train_df['Cuisines'], test_df['Cuisines'],',')

In [11]:
#A function to find all the non numeric values
def non_numerals(series):
    non_numerals = []
    for i in series.unique():
        try :
            i = float(i)
        except:
            non_numerals.append(i)
    return non_numerals

In [12]:
print(non_numerals(train_df['Rating']))
print(non_numerals(train_df['Votes']))
print(non_numerals(train_df['Reviews']))

['-', 'NEW', 'Opening Soon', 'Temporarily Closed']
['-']
['-']


In [13]:
# A function to replace the non-numeric values
def replace_nn_with(series, type_, fill_with = None, method = 'mean'):
    nn = non_numerals(series)
    print('-'*30)
    print('-'*30)
    print("Non Numerals in column ",series.name," : ",nn)
    series = series.replace(nn, np.nan, inplace = False)
    nulls = series.isnull().sum()
    if fill_with:
        series.fillna(fill_with, inplace = True)
        print("Filling Non Numerals with {}".format(fill_with))
    else:
        series = series.replace(nn, np.nan, inplace = False)

    if method == 'mean' :
        rep = series.astype(float).mean()
        print("Filling Non Numerals with MEAN = ", rep)

    elif method =='median' :
        rep = series.astype(float).median()
        print("Filling Non Numerals with MEDIAN = ", rep)

    elif method =='min' :
        rep = series.astype(float).min()
        print("Filling Non Numerals with MINIMUM = ", rep)
    else:
        print('Please pass a valid method as a string -- ("mean" or "median" or "min")')
    return 0

    series.fillna(rep, inplace = True)
    try:
        series = series.astype(type_)
        print(nulls, ": observations replaced")
        return series
    except:
        series =  series.astype(float)
        print(nulls, ": observations replaced")
        series =  series.astype(type_)
    return series

In [14]:
replace_nn_with(train_df['Rating'], float, fill_with = 3.611078, method = 'mean')
replace_nn_with(train_df['Votes'], float, fill_with = '244', method = 'mean')
replace_nn_with(train_df['Reviews'], float, fill_with = 123, method = 'mean')

------------------------------
------------------------------
Non Numerals in column  Rating  :  ['-', 'NEW', 'Opening Soon', 'Temporarily Closed']
Filling Non Numerals with 3.611078
Filling Non Numerals with MEAN =  3.6130382291330485
------------------------------
------------------------------
Non Numerals in column  Votes  :  ['-']
Filling Non Numerals with 244
Filling Non Numerals with MEAN =  244.44267171444025
------------------------------
------------------------------
Non Numerals in column  Reviews  :  ['-']
Filling Non Numerals with 123
Filling Non Numerals with MEAN =  123.19623219758428


0

In [15]:
train_df['Rating'] = train_df['Rating'].apply(lambda x: x.replace('-', '3.6'))
train_df['Rating'] = train_df['Rating'].apply(lambda x: x.replace('NEW', '3.6'))
train_df['Rating'] = train_df['Rating'].apply(lambda x: x.replace('Opening Soon', '3.6'))
train_df['Rating'] = train_df['Rating'].apply(lambda x: x.replace('Temporarily Closed', '3.6'))
train_df['Votes'] = train_df['Votes'].apply(lambda x: x.replace('-', '244'))
train_df['Reviews'] = train_df['Reviews'].apply(lambda x: x.replace('-', '123'))
train_df['Average_Cost'] = train_df['Average_Cost'].apply(lambda x: x.replace('₹', ''))

In [16]:
train_df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time
0,ID_6321,"FTI College, Law College Road, Pune","Fast Food, Rolls, Burger, Salad, Wraps",200,₹50,3.5,12,4,30 minutes
1,ID_2882,"Sector 3, Marathalli","Ice Cream, Desserts",100,₹50,3.5,11,4,30 minutes
2,ID_1595,Mumbai Central,"Italian, Street Food, Fast Food",150,₹50,3.6,99,30,65 minutes
3,ID_5929,"Sector 1, Noida","Mughlai, North Indian, Chinese",250,₹99,3.7,176,95,30 minutes
4,ID_6123,"Rmz Centennial, I Gate, Whitefield","Cafe, Beverages",200,₹99,3.2,521,235,65 minutes


## 전처리

In [17]:
import re

trim_function = lambda x : re.findall("^\s*(.*?)\s*$",str(x))[0]

remove_commas = lambda x: re.sub("[^\d]", "", str(x))


train_df['Average_Cost']= train_df['Average_Cost'].apply(trim_function).apply(remove_commas)

In [18]:
train_df['Average_Cost']=train_df['Average_Cost'].replace(r'^\s*$', np.nan, regex=True)
train_df['Average_Cost']= train_df['Average_Cost'].fillna(train_df['Average_Cost'].value_counts().idxmax())



In [19]:
train_df['Average_Cost']= train_df['Average_Cost'].astype(int)

In [20]:
# test_df['Average_Cost']= test_df['Average_Cost'].apply(trim_function).apply(remove_commas).astype(int)
train_df['Minimum_Order'] = train_df['Minimum_Order'].apply(lambda x: x.replace('₹', ''))
# test_df['Minimum_Order'] = test_df['Minimum_Order'].apply(lambda x: x.replace('₹', ''))


In [21]:
import re

trim_function = lambda x : re.findall("^\s*(.*?)\s*$",str(x))[0]

remove_commas = lambda x: re.sub("[^\d]", "", str(x))


train_df['Minimum_Order']= train_df['Minimum_Order'].apply(trim_function).apply(remove_commas).astype(int)
# test_df['Minimum_Order']= test_df['Minimum_Order'].apply(trim_function).apply(remove_commas).astype(int)

In [22]:
train_df['Rating']= train_df['Rating'].astype(float)
train_df['Votes']= train_df['Votes'].astype(float)
train_df['Reviews']= train_df['Reviews'].astype(float)
# test_df['Rating']= test_df['Rating'].astype(float)
# test_df['Votes']= test_df['Votes'].astype(float)
# test_df['Reviews']= test_df['Reviews'].astype(float)

In [23]:
#Number of words in Location
train_df["sync_num_words"]=train_df['Location'].apply(lambda x :len(str(x).split()))
# test_df["sync_num_words"]=test_df['Location'].apply(lambda x :len(str(x).split()))

#Number of unique words in Location
train_df["Syn_num_unique_words"] = train_df['Location'].apply(lambda x: len(set(str(x).split())))
# test_df["Syn_num_unique_words"] = test_df['Location'].apply(lambda x: len(set(str(x).split())))

#Number of characters in Location

train_df["Syn_num_chars"] = train_df['Location'].apply(lambda x: len(str(x)))
# test_df["Syn_num_chars"] = test_df['Location'].apply(lambda x: len(str(x)))

## Number of stopwords in the Location ##
train_df["Syn_num_stopwords"] = train_df['Location'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))
# test_df["Syn_num_stopwords"] = test_df['Location'].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))

## Number of punctuations in the Location ##
train_df["Syn_num_punctuations"] =train_df['Location'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
# test_df["Syn_num_punctuations"] =test_df['Location'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

## Number of title case words in the Location ##
train_df["Syn_num_words_upper"] = train_df['Location'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
# test_df["Syn_num_words_upper"] = test_df['Location'].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## Number of title case words in the Location ##
train_df["Syn_num_words_title"] = train_df['Location'].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
# test_df["Syn_num_words_title"] = test_df['Location'].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

## Average length of the words in the Location ##
train_df["mean_word_len"] = train_df['Location'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
# test_df["mean_word_len"] = test_df['Location'].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [24]:
def clean_text(text):
    text=text.lower()
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)
    text = re.sub(r'www.[^ ]+', '', text)
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', '', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = [token for token in text.split() if len(token) > 2]
    text = ' '.join(text)
    return text

train_df['Location'] = train_df['Location'].apply(clean_text)
# test_df['Location'] = test_df['Location'].apply(clean_text)

In [25]:
def clean_text(text):
    text=text.lower()
    text = re.sub(r'@[a-zA-Z0-9_]+', '', text)
    text = re.sub(r'https?://[A-Za-z0-9./]+', '', text)
    text = re.sub(r'www.[^ ]+', '', text)
    text = re.sub(r'[a-zA-Z0-9]*www[a-zA-Z0-9]*com[a-zA-Z0-9]*', '', text)
    text = re.sub(r'[^a-zA-Z]', ' ', text)
    text = [token for token in text.split() if len(token) > 2]
    text = ' '.join(text)
    return text

train_df['Cuisines'] = train_df['Cuisines'].apply(clean_text)
# test_df['Cuisines'] = test_df['Cuisines'].apply(clean_text)

In [26]:
## Number of words in the Cuisines ##
train_df["Cuisines_num_words"] = train_df["Cuisines"].apply(lambda x: len(str(x).split()))
# test_df["Cuisines_num_words"] = test_df["Cuisines"].apply(lambda x: len(str(x).split()))

## Number of unique words in the Title ##
train_df["Cuisines_num_unique_words"] = train_df["Cuisines"].apply(lambda x: len(set(str(x).split())))
# test_df["Cuisines_num_unique_words"] = test_df["Cuisines"].apply(lambda x: len(set(str(x).split())))

## Number of characters in the Title ##
train_df["Cuisines_num_chars"] = train_df["Cuisines"].apply(lambda x: len(str(x)))
# test_df["Cuisines_num_chars"] = test_df["Cuisines"].apply(lambda x: len(str(x)))

## Number of stopwords in the Title ##
train_df["Cuisines_num_stopwords"] = train_df["Cuisines"].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))
# test_df["Cuisines_num_stopwords"] = test_df["Cuisines"].apply(lambda x: len([w for w in str(x).lower().split() if w in stop_words]))

## Number of punctuations in the Title ##
train_df["Cuisines_num_punctuations"] =train_df['Cuisines'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )
# test_df["Cuisines_num_punctuations"] =test_df['Cuisines'].apply(lambda x: len([c for c in str(x) if c in string.punctuation]) )

## Number of title case words in the Title ##
train_df["Cuisines_num_words_upper"] = train_df["Cuisines"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))
# test_df["Cuisines_num_words_upper"] = test_df["Cuisines"].apply(lambda x: len([w for w in str(x).split() if w.isupper()]))

## Number of title case words in the Title ##
train_df["Cuisines_num_words_title"] = train_df["Cuisines"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))
# test_df["Cuisines_num_words_title"] = test_df["Cuisines"].apply(lambda x: len([w for w in str(x).split() if w.istitle()]))

## Average length of the words in the Title ##
train_df["mean_word_len_Cuisines"] = train_df["Cuisines"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))
# test_df["mean_word_len_Cuisines"] = test_df["Cuisines"].apply(lambda x: np.mean([len(w) for w in str(x).split()]))

In [27]:
train_df.head()

,Restaurant,Location,Cuisines,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,sync_num_words,Syn_num_unique_words,Syn_num_chars,Syn_num_stopwords,Syn_num_punctuations,Syn_num_words_upper,Syn_num_words_title,mean_word_len,Cuisines_num_words,Cuisines_num_unique_words,Cuisines_num_chars,Cuisines_num_stopwords,Cuisines_num_punctuations,Cuisines_num_words_upper,Cuisines_num_words_title,mean_word_len_Cuisines
0,ID_6321,fti college law college road pune,fast food rolls burger salad wraps,200,50,3.5,12.0,4.0,30 minutes,6,6,35,0,2,1,5,5.000000,6,6,34,0,0,0,0,4.833333
1,ID_2882,sector marathalli,ice cream desserts,100,50,3.5,11.0,4.0,30 minutes,3,3,20,0,1,0,2,6.000000,3,3,18,0,0,0,0,5.333333
2,ID_1595,mumbai central,italian street food fast food,150,50,3.6,99.0,30.0,65 minutes,2,2,14,0,0,0,2,6.500000,5,4,29,0,0,0,0,5.000000
3,ID_5929,sector noida,mughlai north indian chinese,250,99,3.7,176.0,95.0,30 minutes,3,3,15,0,1,0,2,4.333333,4,4,28,0,0,0,0,6.250000
4,ID_6123,rmz centennial gate whitefield,cafe beverages,200,99,3.2,521.0,235.0,65 minutes,5,5,34,1,2,1,5,6.000000,2,2,14,0,0,0,0,6.500000


## Data Preprocessing

In [28]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 1), lowercase=False)
train_route = tf.fit_transform(train_df['Cuisines'])
# test_route = tf.transform(test_df['Cuisines'])

In [29]:
train_route = pd.DataFrame(data=train_route.toarray(), columns=tf.get_feature_names())
# test_route = pd.DataFrame(data=test_route.toarray(), columns=tf.get_feature_names())

In [30]:
train_route.head()

,afghan,african,american,andhra,arabian,asian,assamese,awadhi,bakery,bangladeshi,bar,bbq,belgian,bengali,beverages,bihari,biryani,bohri,brazilian,bubble,burger,burmese,cafe,cantonese,charcoal,chettinad,chicken,chinese,coffee,continental,cream,desserts,dogs,eastern,european,fast,finger,food,french,frozen,german,goan,greek,gujarati,healthy,hot,hyderabadi,ice,indian,indonesian,...,konkan,korean,lankan,lebanese,lucknowi,maharashtrian,malaysian,malwani,mangalorean,meats,mediterranean,mex,mexican,middle,mishti,mithai,modern,momos,mughlai,naga,nepalese,north,odia,paan,parsi,pizza,pok,portuguese,rajasthani,raw,roast,rolls,salad,sandwich,seafood,south,spanish,sri,steak,street,sushi,tamil,tea,tex,thai,tibetan,turkish,vietnamese,wraps,yogurt
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.439374,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.222518,0.0,0.197944,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400383,0.473561,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.577657,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.628052,0.459459,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.628052,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.321313,0.0,0.571656,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.515554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.447136,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.35698,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.727532,0.0,0.0,0.378592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.591743,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.806127,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.00000,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [31]:
train_df = pd.concat([train_df, train_route], axis=1) 
train_df.drop('Cuisines', axis=1, inplace=True)

# test_df = pd.concat([test_df, test_route], axis=1) 
# test_df.drop('Cuisines', axis=1, inplace=True)

In [32]:
train_df.head()

,Restaurant,Location,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,sync_num_words,Syn_num_unique_words,Syn_num_chars,Syn_num_stopwords,Syn_num_punctuations,Syn_num_words_upper,Syn_num_words_title,mean_word_len,Cuisines_num_words,Cuisines_num_unique_words,Cuisines_num_chars,Cuisines_num_stopwords,Cuisines_num_punctuations,Cuisines_num_words_upper,Cuisines_num_words_title,mean_word_len_Cuisines,afghan,african,american,andhra,arabian,asian,assamese,awadhi,bakery,bangladeshi,bar,bbq,belgian,bengali,beverages,bihari,biryani,bohri,brazilian,bubble,burger,burmese,cafe,cantonese,charcoal,chettinad,...,konkan,korean,lankan,lebanese,lucknowi,maharashtrian,malaysian,malwani,mangalorean,meats,mediterranean,mex,mexican,middle,mishti,mithai,modern,momos,mughlai,naga,nepalese,north,odia,paan,parsi,pizza,pok,portuguese,rajasthani,raw,roast,rolls,salad,sandwich,seafood,south,spanish,sri,steak,street,sushi,tamil,tea,tex,thai,tibetan,turkish,vietnamese,wraps,yogurt
0,ID_6321,fti college law college road pune,200,50,3.5,12.0,4.0,30 minutes,6,6,35,0,2,1,5,5.000000,6,6,34,0,0,0,0,4.833333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.439374,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.400383,0.473561,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.577657,0.0
1,ID_2882,sector marathalli,100,50,3.5,11.0,4.0,30 minutes,3,3,20,0,1,0,2,6.000000,3,3,18,0,0,0,0,5.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,ID_1595,mumbai central,150,50,3.6,99.0,30.0,65 minutes,2,2,14,0,0,0,2,6.500000,5,4,29,0,0,0,0,5.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.515554,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,ID_5929,sector noida,250,99,3.7,176.0,95.0,30 minutes,3,3,15,0,1,0,2,4.333333,4,4,28,0,0,0,0,6.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.727532,0.0,0.0,0.378592,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,ID_6123,rmz centennial gate whitefield,200,99,3.2,521.0,235.0,65 minutes,5,5,34,1,2,1,5,6.000000,2,2,14,0,0,0,0,6.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.591743,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.806127,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0


In [33]:
from sklearn.feature_extraction.text import TfidfVectorizer
tf = TfidfVectorizer(ngram_range=(1, 1), lowercase=False)
train_route = tf.fit_transform(train_df['Location'])
# test_route = tf.transform(test_df['Location'])

In [34]:
train_route = pd.DataFrame(data=train_route.toarray(), columns=tf.get_feature_names())
# test_route = pd.DataFrame(data=test_route.toarray(), columns=tf.get_feature_names())
train_route.head()

,administration,area,babarpur,bangalore,bazaar,bazar,begumpet,block,btm,cantt,centennial,central,chandni,chatta,chowk,city,college,colony,court,cst,delhi,dockyard,dumdum,electronic,flats,fti,gate,gora,gtb,gurgaon,gurgoan,high,hyderabad,india,industrial,jaya,kolkata,law,laxman,layout,maharashtra,majestic,malakpet,mangaldas,marathalli,mico,moulali,mumbai,musi,nagar,nathan,new,noida,noorkhan,north,panjetan,public,pune,raja,rajbari,rmz,road,saidabad,sandhurst,school,sector,stage,tejas,timarpur,tiretti,university,vihar,wadala,west,whitefield,yerawada
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.758438,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.379219,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.379219,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.263713,0.0,0.0,0.000000,0.260024,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.891413,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.453193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.794294,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.607534,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.765984,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.642860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.518651,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.439322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.518651,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.518651,0.0


In [35]:
train_df = pd.concat([train_df, train_route], axis=1) 
train_df.drop('Location', axis=1, inplace=True)

# test_df = pd.concat([test_df, test_route], axis=1) 
# test_df.drop('Location', axis=1, inplace=True)

In [36]:
train_df.head()

,Restaurant,Average_Cost,Minimum_Order,Rating,Votes,Reviews,Delivery_Time,sync_num_words,Syn_num_unique_words,Syn_num_chars,Syn_num_stopwords,Syn_num_punctuations,Syn_num_words_upper,Syn_num_words_title,mean_word_len,Cuisines_num_words,Cuisines_num_unique_words,Cuisines_num_chars,Cuisines_num_stopwords,Cuisines_num_punctuations,Cuisines_num_words_upper,Cuisines_num_words_title,mean_word_len_Cuisines,afghan,african,american,andhra,arabian,asian,assamese,awadhi,bakery,bangladeshi,bar,bbq,belgian,bengali,beverages,bihari,biryani,bohri,brazilian,bubble,burger,burmese,cafe,cantonese,charcoal,chettinad,chicken,...,gate,gora,gtb,gurgaon,gurgoan,high,hyderabad,india,industrial,jaya,kolkata,law,laxman,layout,maharashtra,majestic,malakpet,mangaldas,marathalli,mico,moulali,mumbai,musi,nagar,nathan,new,noida,noorkhan,north,panjetan,public,pune,raja,rajbari,rmz,road,saidabad,sandhurst,school,sector,stage,tejas,timarpur,tiretti,university,vihar,wadala,west,whitefield,yerawada
0,ID_6321,200,50,3.5,12.0,4.0,30 minutes,6,6,35,0,2,1,5,5.000000,6,6,34,0,0,0,0,4.833333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.439374,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.379219,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.263713,0.0,0.0,0.000000,0.260024,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,ID_2882,100,50,3.5,11.0,4.0,30 minutes,3,3,20,0,1,0,2,6.000000,3,3,18,0,0,0,0,5.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.891413,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.453193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,ID_1595,150,50,3.6,99.0,30.0,65 minutes,2,2,14,0,0,0,2,6.500000,5,4,29,0,0,0,0,5.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.607534,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,ID_5929,250,99,3.7,176.0,95.0,30 minutes,3,3,15,0,1,0,2,4.333333,4,4,28,0,0,0,0,6.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.765984,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.642860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,ID_6123,200,99,3.2,521.0,235.0,65 minutes,5,5,34,1,2,1,5,6.000000,2,2,14,0,0,0,0,6.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.591743,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.806127,0.0,0.0,0.0,0.0,...,0.439322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.518651,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.518651,0.0


In [37]:
train_df.drop('Restaurant', axis=1, inplace=True)
# test_df.drop('Restaurant', axis=1, inplace=True)
# duplicate_columns = test_df.columns[test_df.columns.duplicated()]
duplicate_columns_1 = train_df.columns[train_df.columns.duplicated()]
train_df.drop('north', axis=1, inplace=True)
# test_df.drop('north', axis=1, inplace=True)

In [38]:
train_df['Delivery_Time'].value_counts()

30 minutes     7406
45 minutes     2665
65 minutes      923
120 minutes      62
20 minutes       20
80 minutes       14
10 minutes        4
Name: Delivery_Time, dtype: int64

In [39]:
X = train_df.drop(labels=['Delivery_Time'], axis=1)
y = train_df['Delivery_Time'].values

from sklearn.model_selection import train_test_split
X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=1)
print("X_train: ",X_train.shape )
print("y_train: ",y_train.shape )
print("X_cv: ",X_cv.shape )
print("y_cv: ",y_cv.shape )

X_train:  (8320, 203)
y_train:  (8320,)
X_cv:  (2774, 203)
y_cv:  (2774,)


In [40]:
X_test = test_df

## XGBoost 학습

In [41]:
from xgboost import XGBClassifier
xgb = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

#Training the classifier
xgb.fit(X_train,y_train)

#Evaluating the score on validation set
xgb.score(X_cv,y_cv)

[12:25:28] WARNING: ../src/learner.cc:516: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




0.7098053352559481

In [42]:
# X_test = X_cv.drop(columns=['class'], axis=1)
X_test = X_cv

from sklearn.metrics import accuracy_score, f1_score
#Predicting for test set
Predictions = xgb.predict(X_test)
f1_score = f1_score(y_cv, Predictions, average='macro')
print("f1_score: %.2f%%" % f1_score)

f1_score: 0.27%


## XGBoost with Normalization by Gonsoo

In [43]:
X2 = X.copy()
y2 = y.copy()

In [44]:
X2.head()

,Average_Cost,Minimum_Order,Rating,Votes,Reviews,sync_num_words,Syn_num_unique_words,Syn_num_chars,Syn_num_stopwords,Syn_num_punctuations,Syn_num_words_upper,Syn_num_words_title,mean_word_len,Cuisines_num_words,Cuisines_num_unique_words,Cuisines_num_chars,Cuisines_num_stopwords,Cuisines_num_punctuations,Cuisines_num_words_upper,Cuisines_num_words_title,mean_word_len_Cuisines,afghan,african,american,andhra,arabian,asian,assamese,awadhi,bakery,bangladeshi,bar,bbq,belgian,bengali,beverages,bihari,biryani,bohri,brazilian,bubble,burger,burmese,cafe,cantonese,charcoal,chettinad,chicken,chinese,coffee,...,fti,gate,gora,gtb,gurgaon,gurgoan,high,hyderabad,india,industrial,jaya,kolkata,law,laxman,layout,maharashtra,majestic,malakpet,mangaldas,marathalli,mico,moulali,mumbai,musi,nagar,nathan,new,noida,noorkhan,panjetan,public,pune,raja,rajbari,rmz,road,saidabad,sandhurst,school,sector,stage,tejas,timarpur,tiretti,university,vihar,wadala,west,whitefield,yerawada
0,200,50,3.5,12.0,4.0,6,6,35,0,2,1,5,5.000000,6,6,34,0,0,0,0,4.833333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.439374,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.379219,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.379219,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.263713,0.0,0.0,0.000000,0.260024,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
1,100,50,3.5,11.0,4.0,3,3,20,0,1,0,2,6.000000,3,3,18,0,0,0,0,5.333333,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.891413,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.453193,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
2,150,50,3.6,99.0,30.0,2,2,14,0,0,0,2,6.500000,5,4,29,0,0,0,0,5.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.607534,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
3,250,99,3.7,176.0,95.0,3,3,15,0,1,0,2,4.333333,4,4,28,0,0,0,0,6.250000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.000000,0.0,0.0,0.0,0.0,0.447136,0.0,...,0.000000,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.765984,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.000000,0.0,0.0,0.0,0.642860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0
4,200,99,3.2,521.0,235.0,5,5,34,1,2,1,5,6.000000,2,2,14,0,0,0,0,6.500000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.591743,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.806127,0.0,0.0,0.0,0.0,0.000000,0.0,...,0.000000,0.439322,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.518651,0.000000,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.518651,0.0


In [45]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_s = scaler.fit_transform(X2)

In [46]:
from sklearn.model_selection import train_test_split
X_s_train, X_s_cv, y_s_train, y_s_cv = train_test_split(X_s, y2, test_size=0.25, random_state=1)

In [47]:
from xgboost import XGBClassifier
xgb_s = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

#Training the classifier
xgb_s.fit(X_s_train,y_s_train)

#Evaluating the score on validation set
xgb_s.score(X_s_cv,y_s_cv)

[12:25:44] WARNING: ../src/learner.cc:516: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




0.7098053352559481

In [48]:
# X_test = X_cv.drop(columns=['class'], axis=1)
X_test = X_s_cv

from sklearn.metrics import accuracy_score, f1_score
#Predicting for test set
Predictions = xgb_s.predict(X_test)
f1_score = f1_score(y_s_cv, Predictions, average='macro')
print("f1_score: %.2f%%" % f1_score)

f1_score: 0.27%


## Remove tfidf

In [49]:
train_cols = X.columns[0:21]
# train_cols = X.columns[202:]

In [50]:
X3 = X[train_cols]
# X3 = X

In [51]:
from sklearn.model_selection import train_test_split
X_s_train, X_s_cv, y_s_train, y_s_cv = train_test_split(X3, y2, test_size=0.25, random_state=1)
print("X_s_train: ", X_s_train.shape)
print("y_s_train: ", y_s_train.shape)
print("X_s_cv: ", X_s_cv.shape)
print("y_s_cv: ", y_s_cv.shape)

X_s_train:  (8320, 21)
y_s_train:  (8320,)
X_s_cv:  (2774, 21)
y_s_cv:  (2774,)


In [52]:
from xgboost import XGBClassifier
xgb_s = XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=1, gamma=0,
              learning_rate=0.1, max_delta_step=0, max_depth=3,
              min_child_weight=1, missing=None, n_estimators=100, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=0,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1, verbosity=1)

#Training the classifier
xgb_s.fit(X_s_train,y_s_train)

#Evaluating the score on validation set
xgb_s.score(X_s_cv,y_s_cv)

[12:26:01] WARNING: ../src/learner.cc:516: 
Parameters: { scale_pos_weight } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




0.7080028839221341

In [53]:
from sklearn.metrics import accuracy_score, f1_score

#Predicting for test set
Predictions = xgb_s.predict(X_s_cv)
Predictions, y_s_cv


(array(['30 minutes', '30 minutes', '30 minutes', ..., '30 minutes',
        '45 minutes', '30 minutes'], dtype=object),
 array(['45 minutes', '30 minutes', '30 minutes', ..., '45 minutes',
        '45 minutes', '30 minutes'], dtype=object))

In [54]:
# evaluate predictions
accuracy = accuracy_score(y_s_cv, Predictions)
print("Accuracy: %.2f%%" % (accuracy * 100.0))
f1_score = f1_score(y_s_cv, Predictions, average='macro')
print("f1_score: %.2f%%" % f1_score)

Accuracy: 70.80%
f1_score: 0.20%


## AutoGluon

In [55]:
import autogluon.core as ag
from autogluon.tabular import TabularPrediction as task

In [56]:
print("X_train: ",X_train.shape )
print("y_train: ",y_train.shape )
print("X_cv: ",X_cv.shape )
print("y_cv: ",y_cv.shape )

X_train:  (8320, 203)
y_train:  (8320,)
X_cv:  (2774, 203)
y_cv:  (2774,)


In [57]:
X_train.insert(0, "class", y_train)
train_df = X_train 

In [58]:
X_cv.insert(0, "class", y_cv)
test_df = X_cv

In [59]:


print("train_df: ", train_df.shape)
print("test_df: ", test_df.shape)


train_df:  (8320, 204)
test_df:  (2774, 204)


In [60]:
train_file_name = 'train.csv'
# X_train, X_cv, y_train, y_cv = train_test_split(X, y, test_size=0.25, random_state=1)
train_df.to_csv(train_file_name, index=None)
test_file_name = 'test.csv'
test_df.to_csv(test_file_name, index=None)

In [61]:
test_data = task.Dataset(file_path="test.csv")

Loaded data from: test.csv | Columns = 204 / 204 | Rows = 2774 -> 2774


In [62]:
train_data = task.Dataset(file_path='train.csv')
print(train_data.shape[0])
subsample_size = 500
subsample_size = train_data.shape[0]
train_data = train_data.sample(n=subsample_size, random_state=0)
print(type(train_data))
train_data.head()

Loaded data from: train.csv | Columns = 204 / 204 | Rows = 8320 -> 8320


8320
<class 'autogluon.tabular.task.tabular_prediction.dataset.TabularDataset'>


,class,Average_Cost,Minimum_Order,Rating,Votes,Reviews,sync_num_words,Syn_num_unique_words,Syn_num_chars,Syn_num_stopwords,Syn_num_punctuations,Syn_num_words_upper,Syn_num_words_title,mean_word_len,Cuisines_num_words,Cuisines_num_unique_words,Cuisines_num_chars,Cuisines_num_stopwords,Cuisines_num_punctuations,Cuisines_num_words_upper,Cuisines_num_words_title,mean_word_len_Cuisines,afghan,african,american,andhra,arabian,asian,assamese,awadhi,bakery,bangladeshi,bar,bbq,belgian,bengali,beverages,bihari,biryani,bohri,brazilian,bubble,burger,burmese,cafe,cantonese,charcoal,chettinad,chicken,chinese,...,fti,gate,gora,gtb,gurgaon,gurgoan,high,hyderabad,india,industrial,jaya,kolkata,law,laxman,layout,maharashtra,majestic,malakpet,mangaldas,marathalli,mico,moulali,mumbai,musi,nagar,nathan,new,noida,noorkhan,panjetan,public,pune,raja,rajbari,rmz,road,saidabad,sandhurst,school,sector,stage,tejas,timarpur,tiretti,university,vihar,wadala,west,whitefield,yerawada
1364,45 minutes,200,50,3.8,551.0,161.0,3,3,15,0,1,0,2,4.333333,3,3,18,0,0,0,0,5.333333,0.0,0.0,0.867652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.765984,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.64286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1870,30 minutes,150,50,4.2,168.0,84.0,5,5,28,0,1,0,5,4.800000,7,6,55,0,0,0,0,7.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.408498,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.290575,...,0.0,0.409182,0.0,0.0,0.0,0.0,0.49528,0.0,0.49528,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3932,45 minutes,150,50,3.5,101.0,53.0,3,3,19,0,1,0,3,5.666667,3,3,18,0,0,0,0,5.333333,0.0,0.0,0.867652,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.467777,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.655959,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
7112,30 minutes,250,50,3.5,19.0,3.0,1,1,8,0,0,0,1,8.000000,3,3,20,0,0,0,0,6.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651732,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.00000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
2337,65 minutes,300,99,2.9,10.0,2.0,3,3,15,0,1,0,2,4.333333,3,3,20,0,0,0,0,6.000000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.651732,...,0.0,0.000000,0.0,0.0,0.0,0.0,0.00000,0.0,0.00000,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.765984,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.64286,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [63]:
label_column = 'class'
print("Summary of class variable: \n", train_data[label_column].value_counts())
dir = 'cdt-predictClass'  # specifies folder where to store trained models
predictor = task.fit(train_data=train_data, label=label_column, output_directory=dir)

Beginning AutoGluon training ...
AutoGluon will save models to cdt-predictClass/
AutoGluon Version:  0.0.15b20201017
Train Data Rows:    8320
Train Data Columns: 203
Preprocessing data ...
AutoGluon infers your prediction problem is: 'multiclass' (because dtype of label-column == object).
	7 unique label values:  ['45 minutes', '30 minutes', '65 minutes', '80 minutes', '120 minutes', '20 minutes', '10 minutes']
	If 'multiclass' is not the correct problem_type, please manually specify the problem_type argument in fit() (You may specify problem_type as one of: ['binary', 'multiclass', 'regression'])
Fraction of data from classes with at least 10 examples that will be kept for training models: 0.9996394230769231
Train Data Class Count: 6
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    58789.74 MB
	Train Data (Original)  Memory Usage: 13.57 MB (0.0% of available memory)
	Inferring data type of each feature 

Summary of class variable: 
 30 minutes     5583
45 minutes     1986
65 minutes      670
120 minutes      51
20 minutes       14
80 minutes       13
10 minutes        3
Name: class, dtype: int64


	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Useless Original Features (Count: 5): ['Cuisines_num_stopwords', 'Cuisines_num_punctuations', 'Cuisines_num_words_upper', 'Cuisines_num_words_title', 'bohri']
		These features carry no predictive signal and should be manually investigated.
		This is typically a feature which has the same value for all rows.
		These features do not need to be present at inference time.
	Types of features in original data (raw dtype, special dtypes):
		('float', []) : 186 | ['Rating', 'Votes', 'Reviews', 'mean_word_len', 'mean_word_len_Cuisines', ...]
		('int', [])   :  12 | ['Average_Cost', 'Minimum_Order', 'sync_num_words', 'Syn_num_unique_words', 'Syn_num_chars', ...]
	Types of features in processed data (raw dtype, special dtypes):
		('float', []) : 186 | ['Rating', 'Votes', 'Reviews', 'mean_word_len', 'mean_word_len_Cuisines', ...]
		('int', [])   :  12 | ['Average_Cost', 'Minimum

In [64]:
# test_data = task.Dataset("test.csv")
y_test = test_data[label_column]
test_data_nolab = test_data.drop(labels=[label_column] , axis=1)
print(test_data_nolab.shape)

(2774, 203)


In [65]:
predictor = task.load(dir)
y_pred = predictor.predict(test_data_nolab)
print("Predictions: ", y_pred)
perf = predictor.evaluate_predictions(y_true= y_test, y_pred= y_pred,auxiliary_metrics=True  )


Evaluation: accuracy on test data: 0.768565248738284
Evaluations on test data:
{
    "accuracy": 0.768565248738284,
    "accuracy_score": 0.768565248738284,
    "balanced_accuracy_score": 0.3505429827196643,
    "matthews_corrcoef": 0.49761240670055157
}
/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning:

Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.

Detailed (per-class) classification report:
{
    "10 minutes": {
        "precision": 0.0,
        "recall": 0.0,
        "f1-score": 0.0,
        "support": 1
    },
    "120 minutes": {
        "precision": 0.8,
        "recall": 0.7272727272727273,
        "f1-score": 0.761904761904762,
        "support": 11
    },
    "20 minutes": {
        "precision": 0.0,
        "recall": 0.0,
        "f1-score": 0.0,
        "support": 6
    },
    "30 minutes":

Predictions:  ['30 minutes' '30 minutes' '30 minutes' ... '30 minutes' '30 minutes'
 '30 minutes']


# Reference
Predicting Food Delivery Time -Hackathon by IMS Proschool
- https://medium.com/code-to-express/predicting-food-delivery-time-hackathon-by-ims-proschool-744fd091390d